<a href="https://colab.research.google.com/github/henrygas/tensorflow_2_learn/blob/master/wnd_criteo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Wide and Deep on TensorFlow (notebook style)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
os.chdir("drive/My Drive/app")

In [4]:
!ls

data  models  svm_data	svm_learn.ipynb


Copyright 2016 Google Inc. All Rights Reserved. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at
 http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

# Introduction

This notebook uses the tf.learn API in TensorFlow to answer a yes/no question. This is called a binary classification problem: Given census data about a person such as age, gender, education and occupation (the features), we will try to predict whether or not the person earns more than 50,000 dollars a year (the target label). 

Given an individual's information our model will output a number between 0 and 1, which can be interpreted as the model's certainty that the individual has an annual income of over 50,000 dollars, (1=True, 0=False)


# Imports and constants
First we'll import our libraries and set up some strings for column names. We also print out the version of TensorFlow we are running.

In [19]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import time
%tensorflow_version 1.x
import tensorflow as tf

# tf.logging.set_verbosity(tf.logging.INFO) # Set to INFO for tracking training, default is WARN. ERROR for least messages

print("Using TensorFlow version %s\n" % (tf.__version__))


CONTINUOUS_COLUMNS =  ["I"+str(i) for i in range(1,14)] # 1-13 inclusive
CATEGORICAL_COLUMNS = ["C"+str(i) for i in range(1,27)] # 1-26 inclusive
LABEL_COLUMN = ["clicked"]

TRAIN_DATA_COLUMNS = LABEL_COLUMN + CONTINUOUS_COLUMNS + CATEGORICAL_COLUMNS
# TEST_DATA_COLUMNS = CONTINUOUS_COLUMNS + CATEGORICAL_COLUMNS

FEATURE_COLUMNS = CONTINUOUS_COLUMNS + CATEGORICAL_COLUMNS

print('Feature columns are: ', FEATURE_COLUMNS, '\n')

 # label is 1
sample = [ 0, 2, 11, 5, 10262, 34, 2, 4, 5, 0, 1, 0, 5, "be589b51", "287130e0", "cd7a7a22", "fb7334df", "25c83c98","0" , "6cdb3998", "361384ce", "a73ee510", "3ff10fb2", "5874c9c9", "976cbd4c", "740c210d", "1adce6ef", "310d155b", "07eb8110", "07c540c4", "891589e7", "18259a83", "a458ea53", "a0ab60ca","0" , "32c7478e", "a052b1ed", "9b3e8820", "8967c0d2"]
print("len(sample):{}".format(len(sample)))

# label is 1
sample = [ 0, 127, 1, 3, 1683, 19, 26, 17, 475, 0, 9, 0, 3, "05db9164", "8947f767", "11c9d79e", "52a787c8", "4cf72387", "fbad5c96", "18671b18", "0b153874", "a73ee510", "ceb10289", "77212bd7", "79507c6b", "7203f04e", "07d13a8f", "2c14c412", "49013ffe", "8efede7f", "bd17c3da", "f6a3e43b", "a458ea53", "35cd95c9", "ad3062eb", "c7dc6720", "3fdb382b", "010f6491", "49d68486"]
print("len(sample):{}".format(len(sample)))

print('Columns and data as a dict: ', dict(zip(FEATURE_COLUMNS, sample)), '\n')

Using TensorFlow version 1.15.0

Feature columns are:  ['I1', 'I2', 'I3', 'I4', 'I5', 'I6', 'I7', 'I8', 'I9', 'I10', 'I11', 'I12', 'I13', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21', 'C22', 'C23', 'C24', 'C25', 'C26'] 

len(sample):39
len(sample):39
Columns and data as a dict:  {'I1': 0, 'I2': 127, 'I3': 1, 'I4': 3, 'I5': 1683, 'I6': 19, 'I7': 26, 'I8': 17, 'I9': 475, 'I10': 0, 'I11': 9, 'I12': 0, 'I13': 3, 'C1': '05db9164', 'C2': '8947f767', 'C3': '11c9d79e', 'C4': '52a787c8', 'C5': '4cf72387', 'C6': 'fbad5c96', 'C7': '18671b18', 'C8': '0b153874', 'C9': 'a73ee510', 'C10': 'ceb10289', 'C11': '77212bd7', 'C12': '79507c6b', 'C13': '7203f04e', 'C14': '07d13a8f', 'C15': '2c14c412', 'C16': '49013ffe', 'C17': '8efede7f', 'C18': 'bd17c3da', 'C19': 'f6a3e43b', 'C20': 'a458ea53', 'C21': '35cd95c9', 'C22': 'ad3062eb', 'C23': 'c7dc6720', 'C24': '3fdb382b', 'C25': '010f6491', 'C26': '49d68486'} 



# Input file parsing

This section puts the file into a `Reader` which reads from the file one batch at a time. 

We set up the Tensors to be a dictionary of features mapping from their string name to the tensor value.

Note that the `_input_fn()` function is wrapped, enabling it to be used for different files.

NOTE: This reads from the input file directly via TensorFlow, rather than using an intermediate tool such as pandas to load the entire dataset into memory first. This is done to enable the system to scale to large inputs.

## More about input functions

The input function is how we will feed the input data into the model during training and evaluation. 
The structure that must be returned is a pair, where the first element is a dict of the column names (features) mapped to a tensor of values, and the 2nd element is a tensor of values representing the answers (labels). Recall that a tensor is just a general term for an n-dimensional array.

This could be represented as: `map(column_name => [Tensor of values]) , [Tensor of labels])`

More concretely, for this particular dataset, something like this:

    { 
      'age':            [ 39, 50, 38, 53, 28, … ], 
      'marital_status': [ 'Married-civ-spouse', 'Never-married', 'Widowed', 'Widowed' … ],
       ...
      'gender':           ['Male', 'Female', 'Male', 'Male', 'Female',, … ], 
    } , 
    [ 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1]
    
Additionally, we define which columns of the input data we will treat as categorical vs continuous, using the global `CATEGORICAL_COLUMNS`.

You can try different values for `BATCH_SIZE` to see how they impact your results

### High-level structure of input functions for CSV-style data
1. Queue file(s)
2. Read a batch of data from the next file
3. Create record defaults, generally 0 for continuous values, and "" for categorical. You can use named types if you prefer
4. Decode the CSV and restructure it to be appropriate for the graph's input format
    * `zip()` column headers with the data
    * `pop()` off the label column(s)
    * Remove/pop any unneeded column(s)
    * Run `tf.expand_dims()` on categorical columns
    5. Return the pair: `(feature_dict, label_array)`
    

In [20]:
BATCH_SIZE = 400

def generate_input_fn(filename, batch_size=BATCH_SIZE):
    def _input_fn():
        filename_queue = tf.train.string_input_producer([filename])
        reader = tf.TextLineReader()
        # Reads out batch_size number of lines
        key, value = reader.read_up_to(filename_queue, num_records=batch_size)
        
        # 1 int label, 13 ints, 26 strings
        cont_defaults = [ [0] for i in range(1,14) ]
        cate_defaults = [ [" "] for i in range(1,27) ]
        label_defaults = [ [0] ]
        column_headers = TRAIN_DATA_COLUMNS
        # The label is the first column of the data.
        record_defaults = label_defaults + cont_defaults + cate_defaults

        # Decode CSV data that was just read out. 
        # Note that this does NOT return a dict, 
        # so we will need to zip it up with our headers
        columns = tf.decode_csv(
            value, record_defaults=record_defaults)
        
        # all_columns is a dictionary that maps from column names to tensors of the data.
        all_columns = dict(zip(column_headers, columns))
        
        # Pop and save our labels 
        # dict.pop() returns the popped array of values; exactly what we need!
        labels = all_columns.pop(LABEL_COLUMN[0])
        
        # the remaining columns are our features
        features = all_columns 

        # Sparse categorical features must be represented with an additional dimension. 
        # There is no additional work needed for the Continuous columns; they are the unaltered columns.
        # See docs for tf.SparseTensor for more info
        for feature_name in CATEGORICAL_COLUMNS:
            features[feature_name] = tf.expand_dims(features[feature_name], -1)

        return features, labels

    return _input_fn

print('input function configured')

input function configured


# Create Feature Columns
This section configures the model with the information about the model. There are many parameters here to experiment with to see how they affect the accuracy.

This is the bulk of the time and energy that is often spent on making a machine learning model work, called *feature selection* or *feature engineering*. We choose the features (columns) we will use for training, and apply any additional transformations to them as needed. 

### Sparse Columns
First we build the sparse columns.

Use `sparse_column_with_keys()` for columns that we know all possible values for.

Use `sparse_column_with_hash_bucket()` for columns that we want the the library to automatically map values for us.

In [21]:
# Sparse base columns.
# C1 = tf.contrib.layers.sparse_column_with_hash_bucket('C1', hash_bucket_size=1000)
# C2 = tf.contrib.layers.sparse_column_with_hash_bucket('C2', hash_bucket_size=1000)
# C3 = tf.contrib.layers.sparse_column_with_hash_bucket('C3', hash_bucket_size=1000)
# ...
# Cn = tf.contrib.layers.sparse_column_with_hash_bucket('Cn', hash_bucket_size=1000)
# wide_columns = [C1, C2, C3, ... , Cn]

wide_columns = []
wide_columns_dict = dict()
for name in CATEGORICAL_COLUMNS:
  each_tensor = tf.contrib.layers.sparse_column_with_hash_bucket(
          name, hash_bucket_size=1000)
  wide_columns.append(each_tensor)
  wide_columns_dict[name] = each_tensor

print('Wide/Sparse columns configured')

Wide/Sparse columns configured


### Continuous columns
Second, configure the real-valued columns using `real_valued_column()`. 

In [22]:
# Continuous base columns.
# I1 = tf.contrib.layers.real_valued_column("I1")
# I2 = tf.contrib.layers.real_valued_column("I2")
# I3 = tf.contrib.layers.real_valued_column("I3")
# ...
# In = tf.contrib.layers.real_valued_column("In")
# deep_columns = [I1, I2, I3, ... , In]

deep_columns = []
deep_columns_dict = dict()
for name in CONTINUOUS_COLUMNS:
  each_tensor = tf.contrib.layers.real_valued_column(name)
  deep_columns.append(each_tensor)
  deep_columns_dict[name] = each_tensor

print('deep/continuous columns configured')

deep/continuous columns configured


### Transformations
Now for the interesting stuff. We will employ a couple of techniques to get even more out of the data.
 
* **bucketizing** turns what would have otherwise been a continuous feature into a categorical one. 
* **feature crossing** allows us to compute a model weight for specific pairings across columns, rather than learning them as independently. This essentially encodes related columns together, for situations where having 2 (or more) columns being certain values is meaningful. 

Only categorical features can be crossed. This is one reason why age has been bucketized.

For example, crossing education and occupation would enable the model to learn about: 

    education="Bachelors" AND occupation="Exec-managerial"

or perhaps 

    education="Bachelors" AND occupation="Craft-repair"

We do a few combined features (feature crosses) here. 

Add your own, based on your intuitions about the dataset, to try to improve on the model!

In [12]:
# No known Transformations. Can add some if desired. 
# Examples from other datasets are shown below.

# age_buckets = tf.contrib.layers.bucketized_column(age,
#             boundaries=[ 18, 25, 30, 35, 40, 45, 50, 55, 60, 65 ])
# education_occupation = tf.contrib.layers.crossed_column([education, occupation], 
#                                                        hash_bucket_size=int(1e4))
# age_race_occupation = tf.contrib.layers.crossed_column([age_buckets, race, occupation], 
#                                                        hash_bucket_size=int(1e6))
# country_occupation = tf.contrib.layers.crossed_column([native_country, occupation], 
                                                      #  hash_bucket_size=int(1e4))

print('Transformations complete')

Transformations complete


### Group feature columns into 2 objects

The wide columns are the sparse, categorical columns that we specified, as well as our hashed, bucket, and feature crossed columns. 

The deep columns are composed of embedded categorical columns along with the continuous real-valued columns. **Column embeddings** transform a sparse, categorical tensor into a low-dimensional and dense real-valued vector. The embedding values are also trained along with the rest of the model. For more information about embeddings, see the TensorFlow tutorial on [Vector Representations Words](https://www.tensorflow.org/tutorials/word2vec/), or [Word Embedding](https://en.wikipedia.org/wiki/Word_embedding) on Wikipedia.

The higher the dimension of the embedding is, the more degrees of freedom the model will have to learn the representations of the features. We are starting with an 8-dimension embedding for simplicity, but later you can come back and increase the dimensionality if you wish.



In [23]:
# Wide columns and deep columns.
# wide_columns = [gender, race, native_country,
#       education, occupation, workclass,
#       marital_status, relationship,
#       age_buckets, education_occupation,
#       age_race_occupation, country_occupation]

# deep_columns = [
#   tf.contrib.layers.embedding_column(workclass, dimension=8),
#   tf.contrib.layers.embedding_column(education, dimension=8),
#   tf.contrib.layers.embedding_column(marital_status, dimension=8),
#   tf.contrib.layers.embedding_column(gender, dimension=8),
#   tf.contrib.layers.embedding_column(relationship, dimension=8),
#   tf.contrib.layers.embedding_column(race, dimension=8),
#   tf.contrib.layers.embedding_column(native_country, dimension=8),
#   tf.contrib.layers.embedding_column(occupation, dimension=8),
#   age,
#   education_num,
#   capital_gain,
#   capital_loss,
#   hours_per_week,
# ]

# Embeddings for wide columns into deep columns
for col in wide_columns:
    deep_columns.append(tf.contrib.layers.embedding_column(col, 
                                                           dimension=8))

print('wide and deep columns configured')

wide and deep columns configured


# Create the model

You can train either a "wide" model, a "deep" model, or a "wide and deep" model, using the classifiers below. Try each one and see what kind of results you get.

* **Wide**: Linear Classifier
* **Deep**: Deep Neural Net Classifier
* **Wide & Deep**: Combined Linear and Deep Classifier

The `hidden_units` or `dnn_hidden_units` argument is to specify the size of each layer of the deep portion of the network. For example, `[12, 20, 15]` would create a network with the first layer of size 12, the second layer of size 20, and a third layer of size 15.

In [24]:
def create_model_dir(model_type):
    # Returns something like models/model_WIDE_AND_DEEP_1493043407
    return 'models/model_' + model_type + '_' + str(int(time.time()))

# Specify the desired model_dir 
def get_model(model_type, model_dir):
    print("Model directory = %s" % model_dir)
    
    # There are more options here than shown here. 
    # We are using this to show additional checkpointing for illustrative purposes.
    # In a real system with far more samples, you would 
    #     likely choose to save checkpoints less frequently.
    runconfig = tf.contrib.learn.RunConfig(
        save_checkpoints_secs=None,
        save_checkpoints_steps = 100,
    )
    
    m = None
    
    # Linear Classifier
    if model_type == 'WIDE':
        m = tf.contrib.learn.LinearClassifier(
            model_dir=model_dir, 
            feature_columns=wide_columns)

    # Deep Neural Net Classifier
    if model_type == 'DEEP':
        m = tf.contrib.learn.DNNClassifier(
            model_dir=model_dir,
            feature_columns=deep_columns,
            hidden_units=[100, 50, 25])

    # Combined Linear and Deep Classifier
    if model_type == 'WIDE_AND_DEEP':
        m = tf.contrib.learn.DNNLinearCombinedClassifier(
            model_dir=model_dir,
            linear_feature_columns=wide_columns,
            dnn_feature_columns=deep_columns,
            dnn_hidden_units=[100, 70, 50, 25],
            config=runconfig)
        
    print('estimator built')
    
    return m
    

MODEL_TYPE = 'WIDE_AND_DEEP'
model_dir = create_model_dir(model_type=MODEL_TYPE)
m = get_model(model_type=MODEL_TYPE, model_dir=model_dir)

Model directory = models/model_WIDE_AND_DEEP_1574407307
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fe9a802c518>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': None, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': 100, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': 'models/model_WIDE_AND_DEEP_1574407307', '_session_creation_timeout_secs': 7200}
estimator built


In [25]:
# Showing that canned estimators return an instance of 'Evaluable'

from tensorflow.contrib.learn.python.learn import evaluable
isinstance(m, evaluable.Evaluable)


True

# Fit the model (train it)

Run `fit()` to train the model. You can experiment with the `train_steps` and `BATCH_SIZE` parameters.

This can take some time, depending on the values chosen for `train_steps` and `BATCH_SIZE`.

Our datafile is hosted on Google Cloud Storage; the reader we created at the beginning knows how to read from it.

If you don't want to download a new copy of the dataset each time your script runs, you can download it locally using 

    gsutil cp gs://dataset-uploader/criteo-kaggle/medium_version/train.csv .
    gsutil cp gs://dataset-uploader/criteo-kaggle/medium_version/eval.csv .
    
If you want to download it manually, use

- http://storageapis.google.com/dataset-uploader/criteo-kaggle/medium_version/eval.csv
- http://storageapis.google.com/dataset-uploader/criteo-kaggle/medium_version/train.csv

In [0]:
# Use the cloud or local depending on your preference

# CLOUD
train_file = "gs://dataset-uploader/criteo-kaggle/medium_version/train.csv"
eval_file  = "gs://dataset-uploader/criteo-kaggle/medium_version/eval.csv"

# LOCAL. Update these paths as appropriate
train_file = "data/wnd/train.csv"
eval_file  = "data/wnd/eval.csv"


In [27]:
%%time

# This can be found with
# wc -l train.csv
train_sample_size = 800000
train_steps = train_sample_size/BATCH_SIZE # 8000/40 = 200

m.fit(input_fn=generate_input_fn(train_file, BATCH_SIZE), steps=train_steps)

print('fit done')

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Instructions for updating:
When switching to tf.estimator.Estimator, use tf.estimator.EstimatorSpec. You can use the `estimator_spec` method to create an equivalent one.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Saving checkpoints for 0 into models/model_WIDE_AND_DEEP_1574407307/model.ckpt.
INFO:tensorflow:loss = 486.01004, step = 2
INFO:tensorflow:Saving checkpoints for 102 into models/model_WIDE_AND_DEEP_15744073

# Evaluate the accuracy of the model
Let's see how the model did. We will evaluate all the test data.

In [28]:
%%time

eval_sample_size = 2000 # this can be found with a 'wc -l eval.csv'
batch_size = 40
eval_steps = eval_sample_size/batch_size # 2000/40 = 50

results = m.evaluate(input_fn=generate_input_fn(eval_file, batch_size=batch_size), 
                     steps=eval_steps)
print('evaluate done')

print('Accuracy: %s' % results['accuracy'])
print(results)


INFO:tensorflow:Starting evaluation at 2019-11-22T07:24:20Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from models/model_WIDE_AND_DEEP_1574407307/model.ckpt-2002
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [5/50]
INFO:tensorflow:Evaluation [10/50]
INFO:tensorflow:Evaluation [15/50]
INFO:tensorflow:Evaluation [20/50]
INFO:tensorflow:Evaluation [25/50]
INFO:tensorflow:Evaluation [30/50]
INFO:tensorflow:Evaluation [35/50]
INFO:tensorflow:Evaluation [40/50]
INFO:tensorflow:Evaluation [45/50]
INFO:tensorflow:Evaluation [50/50]
INFO:tensorflow:Finished evaluation at 2019-11-22-07:24:23
INFO:tensorflow:Saving dict for global step 2002: accuracy = 0.787, accuracy/baseline_label_mean = 0.209, accuracy/threshold_0.500000_mean = 0.787, auc = 0.7166985, auc_precision_recall = 0.38554096, global_step = 2002, labels/actual_label_mean = 0.209, labels/prediction_mean = 0.21091554, loss = 0.4684048, precisi

In [29]:
def pred_fn():
    sample = [ 0, 127, 1, 3, 1683, 19, 26, 17, 475, 0, 9, 0, 3, "05db9164", "8947f767", "11c9d79e", "52a787c8", "4cf72387", "fbad5c96", "18671b18", "0b153874", "a73ee510", "ceb10289", "77212bd7", "79507c6b", "7203f04e", "07d13a8f", "2c14c412", "49013ffe", "8efede7f", "bd17c3da", "f6a3e43b", "a458ea53", "35cd95c9", "ad3062eb", "c7dc6720", "3fdb382b", "010f6491", "49d68486"]
    print("len(sample)={}".format(len(sample)))

    sample_dict = dict(zip(FEATURE_COLUMNS, sample))
    
    for feature_name in CATEGORICAL_COLUMNS:
        sample_dict[feature_name] = tf.expand_dims(sample_dict[feature_name], -1)

    for feature_name in CONTINUOUS_COLUMNS:
        sample_dict[feature_name] = tf.constant(sample_dict[feature_name], shape=(1, 1), dtype=tf.int32)
    
    print(sample_dict)

    return sample_dict

# rtn = m.predict(input_fn=pred_fn)
rtn = m.predict_classes(input_fn=pred_fn)
count = 0
while True:
  count = count + 1
  try:
    print("count={}, class={}".format(count, rtn.__next__()))
  except StopIteration as e:
    break

len(sample)=39
{'I1': <tf.Tensor 'Const:0' shape=(1, 1) dtype=int32>, 'I2': <tf.Tensor 'Const_1:0' shape=(1, 1) dtype=int32>, 'I3': <tf.Tensor 'Const_2:0' shape=(1, 1) dtype=int32>, 'I4': <tf.Tensor 'Const_3:0' shape=(1, 1) dtype=int32>, 'I5': <tf.Tensor 'Const_4:0' shape=(1, 1) dtype=int32>, 'I6': <tf.Tensor 'Const_5:0' shape=(1, 1) dtype=int32>, 'I7': <tf.Tensor 'Const_6:0' shape=(1, 1) dtype=int32>, 'I8': <tf.Tensor 'Const_7:0' shape=(1, 1) dtype=int32>, 'I9': <tf.Tensor 'Const_8:0' shape=(1, 1) dtype=int32>, 'I10': <tf.Tensor 'Const_9:0' shape=(1, 1) dtype=int32>, 'I11': <tf.Tensor 'Const_10:0' shape=(1, 1) dtype=int32>, 'I12': <tf.Tensor 'Const_11:0' shape=(1, 1) dtype=int32>, 'I13': <tf.Tensor 'Const_12:0' shape=(1, 1) dtype=int32>, 'C1': <tf.Tensor 'ExpandDims:0' shape=(1,) dtype=string>, 'C2': <tf.Tensor 'ExpandDims_1:0' shape=(1,) dtype=string>, 'C3': <tf.Tensor 'ExpandDims_2:0' shape=(1,) dtype=string>, 'C4': <tf.Tensor 'ExpandDims_3:0' shape=(1,) dtype=string>, 'C5': <tf.Ten

# Export a model optimized for inference
We can upload our trained model to the Cloud Machine Learning Engine's Prediction Service, which will take care of serving our model and scaling it. The code below exports our trained model to a `saved_model.pb` file and a `variables` folder where the trained weights are stored. 

The `export_savedmodel()` function expects a `serving_input_fn()`, which returns the mapping from the data that the Prediction Service passes in to the data that should be fed into the trained TensorFlow prediction graph.

In [0]:
from tensorflow.contrib.learn.python.learn.utils import input_fn_utils

def column_to_dtype(column):
    if column in CATEGORICAL_COLUMNS:
        return tf.string
    else:
        return tf.float32

def serving_input_fn():
    feature_placeholders = {
        column: tf.placeholder(column_to_dtype(column), [None])
        for column in FEATURE_COLUMNS
    }
    # DNNCombinedLinearClassifier expects rank 2 Tensors, but inputs should be
    # rank 1, so that we can provide scalars to the server
    features = {
        key: tf.expand_dims(tensor, -1)
        for key, tensor in feature_placeholders.items()
    }
    
    return input_fn_utils.InputFnOps(
        features, # input into graph
        None,
        feature_placeholders # tensor input converted from request 
    )
    
    

In [32]:
# Manually export
export_folder = m.export_savedmodel(
    export_dir_base = model_dir + '/export',
    serving_input_fn=serving_input_fn
)

print('model exported successfully to {}'.format(export_folder))

Instructions for updating:
Switch to tf.estimator.Exporter and associated utilities.
Instructions for updating:
Switch to tf.estimator.Exporter and associated utilities.
Instructions for updating:
Switch to tf.estimator.Exporter and associated utilities.
Instructions for updating:
Switch to tf.estimator.Exporter and associated utilities.
Instructions for updating:
Switch to tf.estimator.Exporter and associated utilities.
Instructions for updating:
Switch to tf.estimator.Exporter and associated utilities.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Restoring parameters from models/model_WIDE_AND_DEEP_1574407307/model.ckpt-2002
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: models/model_WIDE_AND_DEEP_1574407307/export/temp-1574407582/saved_model.pb
mode

# Using Experiments to manage the training workflow
TensorFlow also offers an "Experiments" framework to help manage your training for you. Using it will take away some of the boilerplate and automate certain actions, but it's important to understand what is happening (what we did above) before using something like this.

Notice that we don't need to create additional wrapper functions here, but rather, we can directly use the various functions we already defined previously. 

In [0]:
from tensorflow.contrib.learn.python.learn import learn_runner
from tensorflow.contrib.learn.python.learn.utils import saved_model_export_utils


# get_model(model_type = 'WIDE_AND_DEEP', model_dir=model_dir)

# output_dir is an arg passed in by the learn_runner.run() call.
def experiment_fn(output_dir):
    
    print(output_dir)
    
    train_input_fn = generate_input_fn(train_file, BATCH_SIZE)
    eval_input_fn = generate_input_fn(eval_file)
    my_model = get_model(model_type=MODEL_TYPE, 
                  model_dir=output_dir)

    experiment = tf.contrib.learn.Experiment(
        my_model,
        train_input_fn=train_input_fn,
        eval_input_fn=eval_input_fn,
        train_steps=1000,
        export_strategies=[saved_model_export_utils.make_export_strategy(
            serving_input_fn,
            default_output_alternative_key=None,
            exports_to_keep=1
        )]
    )
    return experiment

In [18]:
# manually train and eval
%%time

exp = experiment_fn(model_dir)

exp.train_and_evaluate()

models/model_WIDE_AND_DEEP_1574392007
Model directory = models/model_WIDE_AND_DEEP_1574392007
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fdb9ad60240>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': None, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': 100, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': 'models/model_WIDE_AND_DEEP_1574392007', '_session_creation_timeout_secs': 7200}
estimator built
Instructions for updating:
Switch to tf.estimator.E

In [19]:
# Run the experiment

model_dir=create_model_dir(model_type=MODEL_TYPE)
metrics, output_folder = learn_runner.run(experiment_fn, model_dir)

print('Accuracy: {}'.format(metrics['accuracy']))
print('Model exported to {}'.format(output_folder))

Instructions for updating:
Use tf.estimator.train_and_evaluate.
models/model_WIDE_AND_DEEP_1574392146
Model directory = models/model_WIDE_AND_DEEP_1574392146
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fdb95895f28>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': None, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': 100, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': 'models/model_WIDE_AND_DEEP_1574392146', '_session_creation_timeout_secs': 7200}
est

# Conclusions

In this Juypter notebook, we have configured, created, and evaluated a Wide & Deep machine learning model, that combines the powers of a Linear Classifier with a Deep Neural Network, using TensorFlow's Estimator and Experiment classes.

With this working example in your toolbelt, you are ready to explore the wide (and deep) world of machine learning with TensorFlow! Some ideas to help you get going:
* Change the features we used today. Which columns do you think are correlated and should be crossed? Which ones do you think are just adding noise and could be removed to clean up the model?
* Swap in an entirely new dataset! There are many datasets available on the web, or use a dataset you possess! Check out https://archive.ics.uci.edu/ml to find your own dataset. 